In [1]:
push!(LOAD_PATH, "/home/cfpark/Projects/WST3D/DHC/main/")
import DHC_2DUtils
import Plots
import CUDA
import TimerOutputs
import Statistics
import Profile
import BenchmarkTools

In [2]:
hash_gpu=DHC_2DUtils.fink_filter_hash_gpu(1,8)
image=randn(256,256)
@BenchmarkTools.btime res_gpu=DHC_2DUtils.DHC_compute_gpu(image,hash_gpu,doS2=true,doS12=true, doS20=true);

  117.276 ms (327011 allocations: 35.59 MiB)


In [3]:
hash=DHC_2DUtils.fink_filter_hash(1,8)
image=randn(256,256)
@BenchmarkTools.btime res=DHC_2DUtils.DHC_compute(image,hash,doS2=true,doS12=true, doS20=true,FFTthreads=8);

  184.977 ms (651210 allocations: 310.79 MiB)


In [6]:
Plots.plot((res_gpu.-res))
Plots.plot!(ylim=(-1e-11,1e-11),ylabel="difference",xlabel="coeff #")

LoadError: UndefVarError: res_gpu not defined

In [5]:
hash=DHC_2DUtils.fink_filter_bank_3dizer(hash,1)

Dict{Any, Any} with 20 entries:
  "filt_index"     => Vector{CartesianIndex{3}}[[CartesianIndex(13, 31, 34), Ca…
  "2d_J_L"         => Int32[0 0; 1 0; … ; 5 7; 6 0]
  "S2_iso_mat"     => …
  "2d_psi_index"   => Int32[1 7 … 37 43; 2 8 … 38 44; … ; 5 11 … 41 47; 6 12 … …
  "2d_phi_index"   => 49
  "psi_index"      => Int32[1 37 … 217 253; 7 43 … 223 259; … ; 31 67 … 247 283…
  "cz"             => 1
  "2d_pc"          => 1
  "k_value"        => [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
  "npix"           => 256
  "S1_iso_mat"     => …
  "theta_value"    => [0.0, 0.392699, 0.785398, 1.1781, 1.5708, 1.9635, 2.35619…
  "nz"             => 256
  "2d_filt_index"  => Vector{CartesianIndex{2}}[[CartesianIndex(13, 31), Cartes…
  "2d_filt_value"  => [[0.00151097, 0.00151097, 0.000831601, 0.0119073, 0.01802…
  "J_L_K"          => Int32[1 37 … 217 253; 7 43 … 223 259; … ; 31 67 … 247 283…
  "2d_wd"          => [1.0, 1.0, 1.0, 1.0, 1.0, 2.0]
  "2d_fs_center_r" => [64.0, 32.0, 16.0, 8.0, 4.0, 2.0]
  "filt_value"

In [160]:
import BenchmarkTools
import DHC_2DUtils
filt,info=DHC_2DUtils.fink_filter_bank(1,8);
inds,vals=DHC_2DUtils.fink_filter_list(filt);
mms,vals2=DHC_2DUtils.fink_filter_box(filt);
dat=randn(Float32,256,256)
res=zeros(Float32,256,256)
function mmtest1(dat,filt)
    res=dat.*filt
    return res
end

function mmtest2(dat,inds,vals,res)
    @inbounds @simd for ind in inds
        res[ind]=dat[ind]*vals[ind]
    end
    return res
end;

function mmtest3(dat,mm,vals2,res)
    res[mm[1,1]:mm[1,2],mm[2,1]:mm[2,2]].=dat[mm[1,1]:mm[1,2],mm[2,1]:mm[2,2]].*vals2
    return res
end;

In [161]:
test=12;

In [162]:
@BenchmarkTools.btime mmtest1(dat,filt[:,:,test]);

  101.034 μs (7 allocations: 1.00 MiB)


In [163]:
@BenchmarkTools.btime mmtest2(dat,inds[test],vals[test],res);

  213.096 ns (0 allocations: 0 bytes)


In [164]:
@BenchmarkTools.btime mmtest3(dat,mms[:,:,test],vals2[test],res);

  479.831 ns (5 allocations: 336 bytes)
